In [ ]:
%pip install interpret

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_decision_regions

from sklearn.ensemble import RandomForestClassifier

from interpret.blackbox import LimeTabular
from interpret import show

from interpret.blackbox import LimeTabular
from interpret import show

In [ ]:
dane = pd.read_csv("titanic.csv", index_col=0)
dane.head()

,gender,age,class,embarked,fare,sibsp,parch,survived
0,1,42.0,2,3,7.11,0,0,0
1,1,13.0,2,3,20.05,0,2,0
2,1,16.0,2,3,20.05,1,1,0
3,0,39.0,2,3,20.05,1,1,1
4,0,16.0,2,3,7.13,0,0,1


In [ ]:
dane.describe()

,gender,age,class,embarked,fare,sibsp,parch,survived
count,2207.000000,2207.000000,2207.000000,2207.000000,2207.000000,2207.000000,2207.000000,2207.000000
mean,0.778432,30.436339,2.776167,2.430902,19.991638,0.295877,0.228364,0.322157
std,0.415396,12.154171,2.034319,1.013332,43.322805,0.838714,0.692828,0.467409
min,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,22.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,29.000000,2.000000,3.000000,7.150600,0.000000,0.000000,0.000000
75%,1.000000,38.000000,4.000000,3.000000,21.000000,0.000000,0.000000,1.000000
max,1.000000,74.000000,6.000000,3.000000,512.060700,8.000000,9.000000,1.000000


In [ ]:
X_titanic = dane[dane.columns[:-1]].values
y_titanic = dane['survived'].values

In [ ]:
rf_titanic = RandomForestClassifier()
rf_titanic.fit(X_titanic, y_titanic)
y_hat_titatnic = rf_titanic.predict(X_titanic)
print(rf_titanic.score(X_titanic, y_titanic))

0.9207068418667875


Algorytm RandomForestClassifier poprawnie klasyfikuje czy pasażer przeżył wypadek w 92% analizowanych obserwacji. 

In [ ]:
#4: 16 lat, klasa 2, pasazer przeżył
#43: 27 lat, klasa 1, pasazer nie przeżył
#96: 60 lat, klasa 0, pasazer nie przeżył
#500: 45 lat, klasa 2, pasazer przeżył

osoby = X_titanic[[4, 43, 96, 500], :]

In [ ]:
y_osoby = rf_titanic.predict(osoby)

print(osoby, y_osoby)

[[ 0.     16.      2.      3.      7.13    0.      0.    ]
 [ 1.     27.      1.      3.     10.1     0.      0.    ]
 [ 1.     60.      0.      1.     25.1806  0.      0.    ]
 [ 0.     45.      2.      3.     14.0202  1.      0.    ]] [1 0 0 1]


In [ ]:
lime_inter = LimeTabular(predict_fn=rf_titanic.predict_proba, data=X_titanic, random_state=100)

inter_lokal = lime_inter.explain_local(
    osoby, y_osoby, name='lime')
show(inter_lokal)

/usr/local/lib/python3.7/dist-packages/interpret/provider/visualize.py:44: UserWarning: Cloud environment detected (['colab', 'ipython']): viz integration is still experimental.
  detected_envs


>Osoba nr 1 została zaklasyfikowana jako ocalała z
prawdopodobieństwem na poziomie 86,4%. Główny wpływ na taką kwalifikacje miała zmienna sibsp, w mniejszym stopniu przyczyniły się do tego zmienne fare, embarked oraz class. Zmienne sibsp oraz age zbliżają daną obserwację do klasy 1, natomiast zmienne fare, embarked oraz class działają na korzyść klasy 0.

> Osoba nr 2 została za zaklasyfikowana jako zmarła z prawdopodobieństwem na poziomie 99,5%. Główny wpływ na taką kwalifikacje miały zmienne class oraz sibsp. Zmienne fare, embarked, age oraz parch wpływają na klasyfikacje w mniejszym stopniu. Zmienne class, sibsp, age oraz parch zbliżają daną obserwację do klasy 0, natomiast zmienne fare oraz embarked działają na korzyść klasy 1.

>Osoba nr 3 została za zaklasyfikowana jako zmarła z prawdopodobieństwem na poziomie 96%. Główny wpływ na taką kwalifikacje miały zmienne class oraz sibsp. Zmienne age oraz embarked wpływają na klasyfikacje w mniejszym stopniu. Zmienne class, sibsp, embarked, parch oraz fare zbliżają daną obserwację do klasy 0, natomiast zmienna age działają na korzyść klasy 1.

>Osoba nr 4 została zaklasyfikowana jako ocalała z prawdopodobieństwem na poziomie 73,7%. Główny wpływ na taką kwalifikacje miały zmienne fare, sibsp oraz age. Zmienne embarked, parch oraz wpływają na klasyfikacje w mniejszym stopniu class. Mimo, że wszyskie zmienne oprócz parch oraz gender działają na korzyść klasy 0, to nie przeważa to wartości wyrazu wolnego, co pozwala zakwalifikować obserwację do klasy 1 - jest to poprawna kwalifikacja.

>Na podstawie analizy powyższych czterech osób można wywnioskować, że zmienna gender nie ma wpływu na klasyfikowanie obserwacji. Obserwacje klasyfikowane jako survived = 0, mają wieksze prawdopodobieństwo poprawnej klasyfikacji niż te z wartością 1. Zmienna sibsp jest bardzo istotną zmienną dla obu klas. Zmienna class jest istotniejsza dla przypadków sklasyfikowanych jako 0 (nie przeżyła), natomiast dla przypadków sklasyfikowanych jako 1 istotniejszą zmienną jest fare. 